## Geospatial Data Processing and Vegetation Index Calculation

### Description

This Jupyter Notebook aims to construct a pandas DataFrame containing labeled geospatial data enriched with reflectance values and vegetation indices derived from the label source data. The notebook leverages the SpatioTemporal Asset Catalog (STAC) API to access geospatial datasets, and various Python libraries like pystac_client, geopandas, and rasterio to manipulate the data.

### Sections

* **Setting up the Environment**: The notebook starts by importing the required Python libraries and specifying the STAC API endpoint.

* **Data Retrieval from STAC API**: The notebook connects to the STAC API and retrieves labeled geospatial datasets related to the "ai-extensions-svv-dataset-labels" collection.

* **Utility Functions**: This section defines utility functions for calculating normalized differences (NDVI, NDWI1, NDWI2) and reading geojson data from an AWS S3 bucket.

* **Data Sampling and Processing**: Each labeled geospatial item is processed to sample reflectance values from different spectral bands, such as "coastal", "red", "green", "blue", "nir", "nir08", "nir09", "swir16", and "swir22". The notebook then calculates vegetation indices like NDVI, NDWI1, and NDWI2 based on the sampled data.

* **Creating Enriched DataFrame**: The processed data is organized into a pandas DataFrame, which combines the labeled geospatial data with reflectance values and vegetation indices.

* **Data Description and Saving**: The notebook provides a descriptive summary of the enriched DataFrame, including statistics and characteristics of the data. The DataFrame is then saved as a pickle file for further analysis and visualization.

By following the steps in this notebook, users can obtain a comprehensive DataFrame that contains labeled geospatial data with additional valuable information, such as reflectance values and vegetation indices. This DataFrame can serve as a foundation for various geospatial analysis tasks, including land cover classification, vegetation health monitoring, and environmental assessment. Users can adapt the notebook to work with different datasets and explore other geospatial analytics techniques based on their specific requirements.



In [1]:
from pystac_client import Client
from utils import (
    UserSettings,
    get_asset_by_common_name,
    convert_coordinates,
    get_token,
)
from pystac import read_file
from urllib.parse import urlparse
import geopandas as gpd
import pandas as pd
import boto3, botocore
import io
import os
from pystac.stac_io import DefaultStacIO, StacIO
import rasterio
from pystac import Item
from loguru import logger

Establishes a client for the STAC API endpoint using the pystac library. The cat object is created as a client representation of the STAC catalog at the specified stac_endpoint. This enables interaction with the API and exploration of available items and collections within the catalog.

In [2]:
stac_endpoint = "https://ai-extensions-stac.terradue.com"

headers = []
payload = {
    "client_id": "ai-extensions",
    "username": "ai-extensions-user",
    "password": os.environ.get("IAM_PASSWORD"),
    "grant_type": "password",
}
token = get_token(url=os.environ.get("IAM_URL"), **payload)
headers = {"Authorization": f"Bearer {token}"}

cat = Client.open(stac_endpoint, headers=headers, ignore_conformance=True)
cat

/workspace/.envs/env_label/lib/python3.10/site-packages/pystac_client/client.py:177: FutureWarning: The `ignore_conformance` option is deprecated and will be removed in the next major release. Instead use `set_conforms_to` or `add_conforms_to` to control behavior.
  warnings.warn(


<Client id=stac-fastapi>

A search query is performed on the STAC API using the pystac library. The collections variable is set to ["ai-extensions-svv-dataset-labels"], indicating that we are interested in searching for items within the "ai-extensions-svv-dataset-labels" collection.

In [3]:
collections = ["ai-extensions-svv-dataset-labels"]

query = cat.search(collections=collections)

In [4]:
query.get_all_items()

/workspace/.envs/env_label/lib/python3.10/site-packages/pystac_client/item_search.py:846: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(


The list comprehension iterates over each item in the query.item_collection() and retrieves the "labels" asset for each item using the get_assets() method. The result is a list of "labels" assets for all items in the collection that matched the search query.

In [5]:
[item.get_assets()["labels"] for item in query.item_collection()]

[<Asset href=s3://ai-ext-bucket-dev/svv-dataset/S2B_10SFH_20230613_0_L2A/label-S2B_10SFH_20230613_0_L2A.geojson>,
 <Asset href=s3://ai-ext-bucket-dev/svv-dataset/S2B_10SFG_20230613_0_L2A/label-S2B_10SFG_20230613_0_L2A.geojson>,
 <Asset href=s3://ai-ext-bucket-dev/svv-dataset/S2A_10SFH_20230618_0_L2A/label-S2A_10SFH_20230618_0_L2A.geojson>,
 <Asset href=s3://ai-ext-bucket-dev/svv-dataset/S2A_11SKB_20230618_0_L2A/label-S2A_11SKB_20230618_0_L2A.geojson>,
 <Asset href=s3://ai-ext-bucket-dev/svv-dataset/S2A_10SGG_20230618_0_L2A/label-S2A_10SGG_20230618_0_L2A.geojson>,
 <Asset href=s3://ai-ext-bucket-dev/svv-dataset/S2A_10SFG_20230618_0_L2A/label-S2A_10SFG_20230618_0_L2A.geojson>]

The code sets up user settings using a JSON file named "usersettings.json" and sets the AWS S3 environment using an asset URL obtained from the first item in the search query result.

`UserSettings("usersettings.json")`: A UserSettings object is created, which reads the user settings from the "usersettings.json" file. This file presumably contains various configurations needed for the script.

`settings.set_s3_environment(...)`: The set_s3_environment() method is called on the settings object, passing the absolute URL of the "labels" asset of the first item in the search query result (`query.item_collection()[0].get_assets()["labels"].get_absolute_href()`). This method presumably sets up the AWS S3 environment using the provided asset URL, which might include authentication credentials and other necessary parameters.

`print(os.environ["AWS_ACCESS_KEY_ID"])`: This line prints the value of the "AWS_ACCESS_KEY_ID" environment variable. Since this variable was likely set during the `set_s3_environment()` call in the UserSettings object, it will display the corresponding AWS Access Key ID value.

In [6]:
settings = UserSettings("/etc/Stars/appsettings.json")
# settings.set_s3_vars("argo-wfs")

# print(os.environ["AWS_ACCESS_KEY_ID"])

settings.set_s3_environment(
    query.item_collection()[0].get_assets()["labels"].get_absolute_href()
)


In [7]:
StacIO.set_default(DefaultStacIO)

In [10]:
# create normalized difference function
def nd(a, b):
    """
    Calculates the normalized difference between two values a and b.

    Parameters:
        a (numeric): The first value.
        b (numeric): The second value.

    Returns:
        float: The normalized difference between a and b.

    Example:
        ndvi_value = nd(nir_value, red_value)
    """
    return (a - b) / (a + b)


def read_geojson(
    label_item: Item, user_settings="/etc/Stars/appsettings.json", asset_key="labels"
):
    """
    Reads GeoJSON data from an S3 bucket.

    Parameters:
        label_item (object): Item representing the GeoJSON data with S3 asset link.
        user_settings (str, optional): Path to the user settings file. Default is "usersettings.json".
        asset_key (str, optional): Key to access the GeoJSON asset within the label_item. Default is "labels".

    Returns:
        geopandas.GeoDataFrame: A GeoDataFrame containing the geometries and attributes from the GeoJSON data.

    Raises:
        botocore.exceptions.NoCredentialsError: If AWS credentials are missing or invalid.
        botocore.exceptions.EndpointConnectionError: If there is an issue connecting to the AWS S3 endpoint.
        botocore.exceptions.BotoCoreError: For general BotoCore errors related to AWS S3 interactions.
        geopandas.errors.GeoPandasError: If there is an error reading the GeoJSON data into a GeoDataFrame.

    Example:
        label_data = read_geojson(label_item, user_settings="path/to/usersettings.json", asset_key="geojson_data")
    """
    settings = UserSettings(user_settings)

    settings.set_s3_environment(label_item.get_assets()[asset_key].get_absolute_href())
    session = botocore.session.Session()

    s3_client = session.create_client(
        service_name="s3",
        region_name=os.environ.get("AWS_REGION"),
        use_ssl=True,
        endpoint_url=os.environ.get("AWS_S3_ENDPOINT"),
        aws_access_key_id=os.environ.get("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
    )

    parsed = urlparse(label_item.get_assets()[asset_key].get_absolute_href())

    bucket = parsed.netloc
    key = parsed.path[1:]

    obj = s3_client.get_object(Bucket=bucket, Key=key)

    return gpd.read_file(io.BytesIO(obj["Body"].read()))


def sample_data(label_item, source_item=None, common_bands=["red", "nir"]):
    """
    Samples raster data at specified geometries and calculates spectral indices for each point.

    Parameters:
        label_item (object): GeoJSON data containing geometries to be sampled.
        source_item (object, optional): Raster data file to be sampled. If not provided, it will be inferred from links in label_item. Default is None.
        common_bands (list, optional): List of band names to be read from the raster file. Default is ["red", "nir"].

    Returns:
        geopandas.GeoDataFrame: A GeoDataFrame containing the sampled raster values and calculated spectral indices.

    Raises:
        FileNotFoundError: If the source_item is not provided and cannot be inferred from links in label_item.
        ValueError: If any of the common_bands are not available in the raster data file.

    Example:
        label_data = read_geojson("path/to/label.geojson")
        raster_data = read_file("path/to/raster.tif")
        sampled_data = sample_data(label_data, source_item=raster_data, common_bands=["red", "nir", "swir16"])
    """
    gdf = read_geojson(label_item)

    if source_item is None:
        source_item = read_file(
            [link.target for link in label_item.get_links() if link.rel in ["source"]][
                0
            ]
        )

    dataset = {}
    for common_band in common_bands:
        logger.info(f"Reading {common_band} band")
        dataset[common_band] = rasterio.open(
            get_asset_by_common_name(source_item, common_band).get_absolute_href()
        )

    def convert_row(row, target_crs):
        "EPSG:4326"
        longitude = row.geometry.x
        latitude = row.geometry.y

        src_crs = "EPSG:4326"

        row["utm_x"], row["utm_y"] = convert_coordinates(
            src_crs, target_crs, longitude, latitude
        )

        return pd.Series(row)

    crs_info = dataset[common_bands[0]].crs
    target_crs = f"EPSG:{crs_info.to_epsg()}"
    gdf = gdf.apply(convert_row, target_crs=target_crs, axis=1)

    points_utm = [(x, y) for x, y in zip(gdf["utm_x"], gdf["utm_y"])]

    for common_band in common_bands:
        logger.info(f"Sampling {common_band} band")
        gdf[common_band] = [
            val[0] / 10000 for val in dataset[common_band].sample(points_utm, 1)
        ]

    if "red" in common_bands and "nir" in common_bands:
        gdf["ndvi"] = nd(gdf["nir"], gdf["red"])
    if "green" in common_bands and "nir" in common_bands:
        gdf["ndwi1"] = nd(gdf["green"], gdf["nir"])
    if "nir" in common_bands and "swir16" in common_bands:
        gdf["ndwi2"] = nd(gdf["nir"], gdf["swir16"])

    return gdf

Performs data sampling from the items retrieved in the search query result (query.item_collection()). The sampled data for each item is appended to a list named `tmp_gdfs``, and finally, all the data is concatenated into a pandas DataFrame named `gdf`.

In [11]:
tmp_gdfs = []

for label_item in query.item_collection():

    sampled_data = sample_data(label_item=label_item, common_bands=["coastal", "red", "green", "blue", "nir", "nir08", "nir09", "swir16", "swir22"])
    
    tmp_gdfs.append(sampled_data)

gdf = pd.concat(tmp_gdfs)


2023-09-20 10:29:30.793 | INFO     | __main__:sample_data:98 - Reading coastal band
2023-09-20 10:29:33.423 | INFO     | __main__:sample_data:98 - Reading red band
2023-09-20 10:29:35.378 | INFO     | __main__:sample_data:98 - Reading green band
2023-09-20 10:29:37.226 | INFO     | __main__:sample_data:98 - Reading blue band
2023-09-20 10:29:39.161 | INFO     | __main__:sample_data:98 - Reading nir band
2023-09-20 10:29:41.008 | INFO     | __main__:sample_data:98 - Reading nir08 band
2023-09-20 10:29:42.858 | INFO     | __main__:sample_data:98 - Reading nir09 band
2023-09-20 10:29:44.708 | INFO     | __main__:sample_data:98 - Reading swir16 band
2023-09-20 10:29:46.739 | INFO     | __main__:sample_data:98 - Reading swir22 band
2023-09-20 10:29:52.191 | INFO     | __main__:sample_data:123 - Sampling coastal band
2023-09-20 10:30:04.476 | INFO     | __main__:sample_data:123 - Sampling red band
2023-09-20 10:31:47.342 | INFO     | __main__:sample_data:123 - Sampling green band
2023-09-20 

The `gdf.describe()` method provides a summary statistics of the DataFrame gdf. It computes descriptive statistics for each numeric column in the DataFrame, including measures like count, mean, standard deviation, minimum, quartiles (25%, 50%, 75%), and maximum.

The exact output of gdf.describe() will depend on the content of the DataFrame and the specific numeric columns it contains. Here is a brief overview of the information that may be included in the output:

* count: The number of non-null values for each numeric column.
* mean: The mean (average) value for each numeric column.
* std: The standard deviation for each numeric column, which measures the spread of the data.
* min: The minimum value for each numeric column.
* 25%: The value at the 25th percentile (1st quartile) for each numeric column.
* 50%: The value at the 50th percentile (2nd quartile or median) for each numeric column.
* 75%: The value at the 75th percentile (3rd quartile) for each numeric column.
* max: The maximum value for each numeric column.

The output of `gdf.describe()`` will give you an overview of the central tendency, spread, and distribution of the data in the DataFrame gdf. This summary can be helpful for initial data exploration and gaining insights into the sampled data obtained from the STAC items.

In [12]:
gdf.describe()

,utm_x,utm_y,coastal,red,green,blue,nir,nir08,nir09,swir16,swir22,ndvi,ndwi1,ndwi2
count,1534.000000,1.534000e+03,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000,1534.000000
mean,606369.687313,4.187560e+06,0.064559,0.109282,0.099238,0.077235,0.194373,0.206784,0.208543,0.193951,0.139377,0.197840,-0.219207,0.010196
std,154536.548895,5.366458e+04,0.050475,0.066830,0.055794,0.055445,0.114833,0.121875,0.117879,0.119755,0.086418,0.335595,0.369345,0.186601
min,200009.978139,4.090250e+06,0.000200,0.003600,0.008800,0.000100,0.000100,0.000100,0.000100,0.003000,0.003100,-0.973333,-0.845459,-0.987097
25%,615859.964959,4.148850e+06,0.036200,0.062425,0.066400,0.045050,0.080400,0.082325,0.090025,0.079525,0.064825,-0.068093,-0.505200,-0.109481
50%,655189.998843,4.186600e+06,0.052700,0.096550,0.089200,0.067000,0.216500,0.234800,0.240450,0.192950,0.129450,0.197909,-0.349640,-0.006430
75%,689600.010583,4.223425e+06,0.074450,0.146000,0.114550,0.091375,0.273400,0.294075,0.290400,0.296575,0.204650,0.431809,0.108895,0.083600
max,753550.029670,4.299910e+06,0.708000,0.742400,0.732800,0.707600,0.729600,0.735400,0.687500,0.617900,0.491000,0.926432,0.991111,0.859485


The `gdf`` DataFrame, which contains the sampled data from the STAC items, is saved as a pickle file named 'sprint-0-STAC-labels-to-dataframe.pkl'. Pickle is a Python serialization format used to save Python objects to disk.

The `to_pickle()` method is used to serialize the DataFrame gdf and store it as a binary pickle file with the specified filename ('sprint-0-STAC-labels-to-dataframe.pkl').

By saving the DataFrame as a pickle file, you can later load it back into memory and access the sampled data without having to recompute or resample it. This is particularly useful when working with large datasets or when you need to share the data with others while preserving its structure and content. To load the pickle file back into a DataFrame, you can use the `pd.read_pickle()` function.

In [ ]:
gdf.to_pickle('sprint-0-STAC-labels-to-dataframe.pkl')